# Caracterización de la Distribución (Comuna)

La presente caracterización se hace a través de *'df_caracterizacion_distribucion'*, el cual corresponde a una tabla que ha sido procesada en '*Caracterizacion_Dx_Dic2024_comuna-empresa (CNE).ipynb*' (normalizando registros, completando nulos y juntando información de diversas fuentes). La fuente de esta base de datos está hecha a partir de la información proporcionada por la Comisión Nacional de Energía a través del Anexo del cálculo del Precio Nudo Promedio (PNP). Los datos de consumo se encuentran actualizados a diciembre del 2024.

La idea de este notebook es poder representar la información que ha sido organizada a través de la comuna y la empresa en '*Caracterizacion_Dx_Dic2024_comuna-empresa (CNE).ipynb*', para pasarla a un esquema definido solo por la comuna.

In [1]:
import pandas as pd 
import numpy as np
import unidecode as ud
from sklearn.linear_model import LinearRegression
from pprint import pprint

## Reorganización de la información (Comuna-empresa) -> (Comuna)

In [2]:
df_caracterizacion_distribucion_ce = pd.read_excel("./Salidas/Caracterizacion_Dx_Dic2024_comuna-empresa (CNE).xlsx")
df_caracterizacion_distribucion_ce

,Distribuidor,Comuna,Segmento,Densidad,Cantidad de clientes,Energía promedio mensual [kWh],Tipo de facturación,Cantidad de alimentadores,UM 1F 1S para TD,UM 3F 1S para TD,UM 3F 2S para TD,Pérdidas de energía [pu],Hurto de energía [pu],Pérdidas comerciales [pu],Energía facturada [CLP]
0,cec,chimbarongo,Residencial,EXTREMADAMENTE BAJA,253,47719.28,Mensual,0,0,0,0,0.0292,0.0388,0.0097,0
1,cec,chimbarongo,No Residencial BT,EXTREMADAMENTE BAJA,1,4668.92,Mensual,0,0,0,0,0.0292,0.0388,0.0097,0
2,cec,chimbarongo,No Residencial AT,EXTREMADAMENTE BAJA,9,50207.19,Mensual,0,0,0,0,0.0292,0.0388,0.0097,0
3,cec,chimbarongo,LibreDx,EXTREMADAMENTE BAJA,0,0.00,Mensual,0,0,0,0,0.0292,0.0388,0.0097,0
4,cec,chimbarongo,Red,EXTREMADAMENTE BAJA,0,0.00,Mensual,4,12,15,52,0.0292,0.0388,0.0097,20805863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,socoepa,panguipulli,Residencial,EXTREMADAMENTE BAJA,523,103431.11,Mensual,0,0,0,0,0.0158,0.0158,0.0158,0
2421,socoepa,panguipulli,No Residencial BT,EXTREMADAMENTE BAJA,3,2864.42,Mensual,0,0,0,0,0.0158,0.0158,0.0158,0
2422,socoepa,panguipulli,No Residencial AT,EXTREMADAMENTE BAJA,19,28984.52,Mensual,0,0,0,0,0.0158,0.0158,0.0158,0
2423,socoepa,panguipulli,LibreDx,EXTREMADAMENTE BAJA,0,0.00,Mensual,0,0,0,0,0.0158,0.0158,0.0158,0


In [3]:
# Se extraen las columnas Densidad y Tipo de facturacion
df_comuna_empresa_densidad = df_caracterizacion_distribucion_ce.groupby(["Distribuidor", "Comuna", "Densidad"], as_index=False)["Cantidad de clientes"].sum()
df_comuna_empresa_facturacion = df_caracterizacion_distribucion_ce.groupby(["Distribuidor", "Comuna", "Tipo de facturación"], as_index=False)["Cantidad de clientes"].sum()

# Se mapean los valores de densidad y facturacion
Mapa_densidad = {
    "EXTREMADAMENTE BAJA": 1,
    "MUY BAJA": 2,
    "BAJA": 3,
    "MEDIA": 4,
    "ALTA": 5}

Mapa_facturacion = {
    "Mensual": 1,
    "Bimensual": 2}

df_comuna_empresa_densidad["Mapeo_Densidad"] = df_comuna_empresa_densidad["Densidad"].map(Mapa_densidad)
df_comuna_empresa_facturacion["Mapeo_facturacion"] = df_comuna_empresa_facturacion["Tipo de facturación"].map(Mapa_facturacion)

# Se actualiza el dataframe original sin las columnas Densidad y Tipo de facturacion
df_caracterizacion_distribucion_ce = df_caracterizacion_distribucion_ce[["Distribuidor", "Comuna", "Segmento", "Cantidad de clientes", "Energía promedio mensual [kWh]",
                                                                         "Cantidad de alimentadores", "UM 1F 1S para TD", "UM 3F 1S para TD", "UM 3F 2S para TD", 
                                                                         "Pérdidas de energía [pu]", "Hurto de energía [pu]", "Pérdidas comerciales [pu]", "Energía facturada [CLP]"]]

columnas_a_sumar = [
    "Cantidad de clientes",
    "Energía promedio mensual [kWh]",
    "Cantidad de alimentadores",
    "UM 1F 1S para TD",
    "UM 3F 1S para TD",
    "UM 3F 2S para TD",
    "Energía facturada [CLP]"
]

# Se agrupa por comuna y segmento y se promedia el Hurto y las Perdidas comerciales
df_caracterizacion_distribucion_c = (df_caracterizacion_distribucion_ce.groupby(["Comuna", "Segmento"], sort=False, as_index=False)
                                     									.agg({**{col: "sum" for col in columnas_a_sumar}, 
                                                    							"Hurto de energía [pu]": "mean",
                                                                                "Pérdidas comerciales [pu]": "mean",
                                                                                "Pérdidas de energía [pu]": "mean"}))

df_caracterizacion_distribucion_c["Hurto de energía [pu]"] = df_caracterizacion_distribucion_c["Hurto de energía [pu]"].round(4)
df_caracterizacion_distribucion_c["Pérdidas de energía [pu]"] = df_caracterizacion_distribucion_c["Pérdidas de energía [pu]"].round(4)
df_caracterizacion_distribucion_c["Energía promedio mensual [kWh]"] = df_caracterizacion_distribucion_c["Energía promedio mensual [kWh]"].round(2)
df_caracterizacion_distribucion_c

,Comuna,Segmento,Cantidad de clientes,Energía promedio mensual [kWh],Cantidad de alimentadores,UM 1F 1S para TD,UM 3F 1S para TD,UM 3F 2S para TD,Energía facturada [CLP],Hurto de energía [pu],Pérdidas comerciales [pu],Pérdidas de energía [pu]
0,chimbarongo,Residencial,13841,2360938.71,0,0,0,0,0,0.0444,0.0115,0.0274
1,chimbarongo,No Residencial BT,407,646085.70,0,0,0,0,0,0.0444,0.0115,0.0274
2,chimbarongo,No Residencial AT,295,2006453.80,0,0,0,0,0,0.0444,0.0115,0.0274
3,chimbarongo,LibreDx,0,0.00,0,0,0,0,0,0.0444,0.0115,0.0274
4,chimbarongo,Red,0,0.00,12,33,66,221,947763849,0.0444,0.0115,0.0274
...,...,...,...,...,...,...,...,...,...,...,...,...
1645,valdivia,Residencial,72207,14252066.32,0,0,0,0,0,0.0164,0.0102,0.0166
1646,valdivia,No Residencial BT,1541,2922896.17,0,0,0,0,0,0.0164,0.0102,0.0166
1647,valdivia,No Residencial AT,563,4384458.50,0,0,0,0,0,0.0164,0.0102,0.0166
1648,valdivia,LibreDx,0,0.00,0,0,0,0,0,0.0164,0.0102,0.0166


In [4]:
Mapa_densidad = {
    1: "EXTREMADAMENTE BAJA",
    2: "MUY BAJA",
    3: "BAJA",
    4: "MEDIA",
    5: "ALTA"}

Mapa_facturacion = {
    1: "Mensual",
    2: "Bimensual"}

# Se calcula la densidad promedio ponderada por cantidad de clientes (Mapeo_Densidad)
densidad_ponderada = (df_comuna_empresa_densidad.assign(_num=lambda d: d['Mapeo_Densidad'] * d['Cantidad de clientes'],
														_den=lambda d: d['Cantidad de clientes'])
												.groupby('Comuna', as_index=False)[['_num','_den']].sum()
												.assign(Mapeo_Densidad_promedio=lambda d: d['_num'] / d['_den'])
												.drop(columns=['_num','_den']))

# Se calcula el tipo de facturacion promedio ponderada por cantidad de clientes (Mapeo_facturacion)
facturacion_ponderada = (df_comuna_empresa_facturacion.assign(_num=lambda d: d['Mapeo_facturacion'] * d['Cantidad de clientes'],
																_den=lambda d: d['Cantidad de clientes'])
														.groupby('Comuna', as_index=False)[['_num','_den']].sum()
														.assign(Mapeo_Facturacion_promedio=lambda d: d['_num'] / d['_den'])
														.drop(columns=['_num','_den']))

# Se unen ambos dataframes, densidad ponderada y facturacion ponderada
df_estimacion = densidad_ponderada.merge(facturacion_ponderada, on="Comuna", how="left")


# Se redondea al entero ambos resultados y se mapea la etiqueta
df_estimacion["Mapeo_Densidad_promedio"] = (df_estimacion["Mapeo_Densidad_promedio"].round(0).clip(1, 5).astype(int))
df_estimacion["Mapeo_Facturacion_promedio"] = (df_estimacion["Mapeo_Facturacion_promedio"].round(0).clip(1, 2).astype(int))

df_estimacion["Densidad"] = df_estimacion["Mapeo_Densidad_promedio"].map(Mapa_densidad)
df_estimacion["Tipo de facturacion"] = df_estimacion["Mapeo_Facturacion_promedio"].map(Mapa_facturacion)
df_estimacion = df_estimacion.drop(columns=["Mapeo_Densidad_promedio", "Mapeo_Facturacion_promedio"])

# Se agrega la informacion de densidad y facturacion al dataframe de caracterizacion por comuna y segmento
df_caracterizacion_distribucion = df_caracterizacion_distribucion_c.merge(df_estimacion[["Comuna", "Densidad", "Tipo de facturacion"]], on="Comuna", how="left")

df_caracterizacion_distribucion = df_caracterizacion_distribucion.rename(columns={"UM 1F 1S para TD": "UM 1F para TD"})
df_caracterizacion_distribucion

,Comuna,Segmento,Cantidad de clientes,Energía promedio mensual [kWh],Cantidad de alimentadores,UM 1F para TD,UM 3F 1S para TD,UM 3F 2S para TD,Energía facturada [CLP],Hurto de energía [pu],Pérdidas comerciales [pu],Pérdidas de energía [pu],Densidad,Tipo de facturacion
0,chimbarongo,Residencial,13841,2360938.71,0,0,0,0,0,0.0444,0.0115,0.0274,MUY BAJA,Mensual
1,chimbarongo,No Residencial BT,407,646085.70,0,0,0,0,0,0.0444,0.0115,0.0274,MUY BAJA,Mensual
2,chimbarongo,No Residencial AT,295,2006453.80,0,0,0,0,0,0.0444,0.0115,0.0274,MUY BAJA,Mensual
3,chimbarongo,LibreDx,0,0.00,0,0,0,0,0,0.0444,0.0115,0.0274,MUY BAJA,Mensual
4,chimbarongo,Red,0,0.00,12,33,66,221,947763849,0.0444,0.0115,0.0274,MUY BAJA,Mensual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645,valdivia,Residencial,72207,14252066.32,0,0,0,0,0,0.0164,0.0102,0.0166,BAJA,Bimensual
1646,valdivia,No Residencial BT,1541,2922896.17,0,0,0,0,0,0.0164,0.0102,0.0166,BAJA,Bimensual
1647,valdivia,No Residencial AT,563,4384458.50,0,0,0,0,0,0.0164,0.0102,0.0166,BAJA,Bimensual
1648,valdivia,LibreDx,0,0.00,0,0,0,0,0,0.0164,0.0102,0.0166,BAJA,Bimensual


### Superficie efectiva por Comuna

In [5]:
df_superficies_por_comuna = pd.read_excel("./Datos_Dx_procesados/Superficie por comuna (procesados).xlsx")
df_superficies_por_comuna

,Comuna,Superficie_efectiva_km2,Largo_red_MT_km,Largo_red_BT_km,Superficie_km2
0,alhue,33.750000,88.366937,68.257696,845.0
1,alto biobio,15.250000,41.413314,28.531563,2125.0
2,alto del carmen,51.750000,187.921759,54.939257,5939.0
3,alto hospicio,22.500000,100.630280,175.026003,573.0
4,ancud,341.000000,745.638398,522.839057,1752.0
...,...,...,...,...,...
325,natales,14194.533231,34815.465783,23416.424784,49924.0
326,porvenir,2759.921763,6798.421636,4611.972004,9707.0
327,punta arenas,5074.025319,12468.429899,8417.562704,17846.0
328,putre,1678.357697,4148.377229,2833.317744,5903.0


In [6]:
# Se combina la informacion de superficie con el dataframe de caracterizacion por comuna y segmento
df_caracterizacion_distribucion = df_caracterizacion_distribucion.merge(df_superficies_por_comuna[["Comuna", "Superficie_efectiva_km2"]], on="Comuna", how="left")

# Se asigna 0 a la columna 'Superficie_efectiva_km2' cuando el segmento no es 'Red'
df_caracterizacion_distribucion.loc[df_caracterizacion_distribucion["Segmento"] != "Red", ["Superficie_efectiva_km2"]] = 0

# Se reordenan las columnas
cols = [
    "Comuna",
    "Segmento",
    "Densidad",
    "Tipo de facturacion",
    "Superficie_efectiva_km2",
    "Cantidad de clientes",
    "Cantidad de alimentadores",
    "UM 1F para TD",
    "UM 3F 1S para TD",
    "UM 3F 2S para TD",
    "Energía promedio mensual [kWh]",
    "Energía facturada [CLP]",
    "Pérdidas de energía [pu]",
    "Hurto de energía [pu]",
    "Pérdidas comerciales [pu]"
]

df_caracterizacion_distribucion = df_caracterizacion_distribucion[cols]

# Se pasa a USD la energía facturada ajustada por CPI
tasa_cambio_clp_usd = 982.30 		# Dic 2024
df_caracterizacion_distribucion["Energía facturada [CLP]"] = (df_caracterizacion_distribucion["Energía facturada [CLP]"] / tasa_cambio_clp_usd) * 1.03   # Ajuste CPI 3%
df_caracterizacion_distribucion = df_caracterizacion_distribucion.rename(columns={"Energía facturada [CLP]": "Energía facturada [USD]"})
df_caracterizacion_distribucion

,Comuna,Segmento,Densidad,Tipo de facturacion,Superficie_efectiva_km2,Cantidad de clientes,Cantidad de alimentadores,UM 1F para TD,UM 3F 1S para TD,UM 3F 2S para TD,Energía promedio mensual [kWh],Energía facturada [USD],Pérdidas de energía [pu],Hurto de energía [pu],Pérdidas comerciales [pu]
0,chimbarongo,Residencial,MUY BAJA,Mensual,0.00,13841,0,0,0,0,2360938.71,0.000000e+00,0.0274,0.0444,0.0115
1,chimbarongo,No Residencial BT,MUY BAJA,Mensual,0.00,407,0,0,0,0,646085.70,0.000000e+00,0.0274,0.0444,0.0115
2,chimbarongo,No Residencial AT,MUY BAJA,Mensual,0.00,295,0,0,0,0,2006453.80,0.000000e+00,0.0274,0.0444,0.0115
3,chimbarongo,LibreDx,MUY BAJA,Mensual,0.00,0,0,0,0,0,0.00,0.000000e+00,0.0274,0.0444,0.0115
4,chimbarongo,Red,MUY BAJA,Mensual,130.00,0,12,33,66,221,0.00,9.937868e+05,0.0274,0.0444,0.0115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645,valdivia,Residencial,BAJA,Bimensual,0.00,72207,0,0,0,0,14252066.32,0.000000e+00,0.0166,0.0164,0.0102
1646,valdivia,No Residencial BT,BAJA,Bimensual,0.00,1541,0,0,0,0,2922896.17,0.000000e+00,0.0166,0.0164,0.0102
1647,valdivia,No Residencial AT,BAJA,Bimensual,0.00,563,0,0,0,0,4384458.50,0.000000e+00,0.0166,0.0164,0.0102
1648,valdivia,LibreDx,BAJA,Bimensual,0.00,0,0,0,0,0,0.00,0.000000e+00,0.0166,0.0164,0.0102


### SPD por Comuna

La cantidad de Subestaciones primarias por Comuna viene de la información de la base de datos del Coordinador. Se pre-procesa en 'Pre-procesamiento_Datos_Dx.ipynb' en la sección df_clientes.

In [7]:
df_se_por_comuna = pd.read_excel("./Datos_Dx_procesados/Cantidad_SE_por_Comuna.xlsx", engine='openpyxl')

df_se_por_comuna.rename(columns={"Cantidad_SE_Primaria": "Cantidad SPD"}, inplace=True)
df_se_por_comuna

,Comuna,Cantidad SPD
0,aisen,1
1,alto biobio,1
2,ancud,2
3,angol,4
4,antuco,1
...,...,...
141,victoria,3
142,vilcun,3
143,villarrica,9
144,yumbel,3


In [8]:
# Se agrega la informacion de cantidad de SE por comuna al dataframe de caracterizacion por comuna y segmento
# Si no se tiene informacion de SE para una comuna, se asigna 0
df_caracterizacion_distribucion = df_caracterizacion_distribucion.merge(df_se_por_comuna[["Comuna", "Cantidad SPD"]],
																		on="Comuna",
																		how="left")

df_caracterizacion_distribucion["Cantidad SPD"] = df_caracterizacion_distribucion["Cantidad SPD"].fillna(0).astype(int)

df_caracterizacion_distribucion.loc[df_caracterizacion_distribucion["Segmento"] != "Red", "Cantidad SPD"] = 0

df_caracterizacion_distribucion

,Comuna,Segmento,Densidad,Tipo de facturacion,Superficie_efectiva_km2,Cantidad de clientes,Cantidad de alimentadores,UM 1F para TD,UM 3F 1S para TD,UM 3F 2S para TD,Energía promedio mensual [kWh],Energía facturada [USD],Pérdidas de energía [pu],Hurto de energía [pu],Pérdidas comerciales [pu],Cantidad SPD
0,chimbarongo,Residencial,MUY BAJA,Mensual,0.00,13841,0,0,0,0,2360938.71,0.000000e+00,0.0274,0.0444,0.0115,0
1,chimbarongo,No Residencial BT,MUY BAJA,Mensual,0.00,407,0,0,0,0,646085.70,0.000000e+00,0.0274,0.0444,0.0115,0
2,chimbarongo,No Residencial AT,MUY BAJA,Mensual,0.00,295,0,0,0,0,2006453.80,0.000000e+00,0.0274,0.0444,0.0115,0
3,chimbarongo,LibreDx,MUY BAJA,Mensual,0.00,0,0,0,0,0,0.00,0.000000e+00,0.0274,0.0444,0.0115,0
4,chimbarongo,Red,MUY BAJA,Mensual,130.00,0,12,33,66,221,0.00,9.937868e+05,0.0274,0.0444,0.0115,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645,valdivia,Residencial,BAJA,Bimensual,0.00,72207,0,0,0,0,14252066.32,0.000000e+00,0.0166,0.0164,0.0102,0
1646,valdivia,No Residencial BT,BAJA,Bimensual,0.00,1541,0,0,0,0,2922896.17,0.000000e+00,0.0166,0.0164,0.0102,0
1647,valdivia,No Residencial AT,BAJA,Bimensual,0.00,563,0,0,0,0,4384458.50,0.000000e+00,0.0166,0.0164,0.0102,0
1648,valdivia,LibreDx,BAJA,Bimensual,0.00,0,0,0,0,0,0.00,0.000000e+00,0.0166,0.0164,0.0102,0


### Resumen de los resultados de la caracterización de la demanda

In [8]:
# Información al mes de diciembre de 2024
# -> Cantidad de clientes.
# -> Cantidad de energía.
cantidad_clientes = df_caracterizacion_distribucion['Cantidad de clientes'].sum()
cantidad_energia = df_caracterizacion_distribucion['Energía promedio mensual [kWh]'].sum()

df_facturacion_clientes_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'Residencial']
cantidad_clientes_residenciales = df_facturacion_clientes_residenciales['Cantidad de clientes'].sum()
cantidad_energia_residenciales = df_facturacion_clientes_residenciales['Energía promedio mensual [kWh]'].sum()

df_facturacion_clientes_no_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'No Residencial BT']
cantidad_clientes_no_residenciales_bt = df_facturacion_clientes_no_residenciales['Cantidad de clientes'].sum()
cantidad_energia_no_residenciales_bt = df_facturacion_clientes_no_residenciales['Energía promedio mensual [kWh]'].sum()

df_facturacion_clientes_no_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'No Residencial AT']
cantidad_clientes_no_residenciales_at = df_facturacion_clientes_no_residenciales['Cantidad de clientes'].sum()
cantidad_energia_no_residenciales_at = df_facturacion_clientes_no_residenciales['Energía promedio mensual [kWh]'].sum()

df_facturacion_clientes_no_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'LibreDx']
cantidad_clientes_no_residenciales_libredx = df_facturacion_clientes_no_residenciales['Cantidad de clientes'].sum()
cantidad_energia_no_residenciales_libredx = df_facturacion_clientes_no_residenciales['Energía promedio mensual [kWh]'].sum()

cantidad_um_1f_td = df_caracterizacion_distribucion['UM 1F para TD'].sum()
cantidad_um_3f1s_td = df_caracterizacion_distribucion['UM 3F 1S para TD'].sum()
cantidad_um_3f2s_td = df_caracterizacion_distribucion['UM 3F 2S para TD'].sum()

print("(A fecha de diciembre de 2024)",
      "\nClientes:", 
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes), 
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia * 12 / 1_000_000_000).round(4)),
      "\nClientes Residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_residenciales),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_residenciales * 12 / 1_000_000_000).round(4)),
      "\nClientes No Residenciales en BT:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales_bt),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_no_residenciales_bt * 12 / 1_000_000_000).round(4)),
      "\nClientes No Residenciales en AT:",
	  "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales_at),
	  "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_no_residenciales_at * 12 / 1_000_000_000).round(4)),
      "\nClientes No Residenciales LibreDx:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales_libredx),
      "\n -> Energía facturada: {:,} TWh".format((cantidad_energia_no_residenciales_libredx * 12 / 1_000_000_000).round(4)),
      "\n--------------------------------------------",
      "\nCantidad de UM para TD:",
	  "\n -> UM 1F para TD: {:,}".format(cantidad_um_1f_td),
	  "\n -> UM 3F 1S para TD: {:,}".format(cantidad_um_3f1s_td),
	  "\n -> UM 3F 2S para TD: {:,}".format(cantidad_um_3f2s_td),
	  "\n")

(A fecha de diciembre de 2024) 
Clientes: 
 -> Cantidad de clientes: 7,628,895 
 -> Energía facturada: 33.9547 TWh 
Clientes Residenciales: 
 -> Cantidad de clientes: 7,429,659 
 -> Energía facturada: 16.4958 TWh 
Clientes No Residenciales en BT: 
 -> Cantidad de clientes: 142,810 
 -> Energía facturada: 4.3814 TWh 
Clientes No Residenciales en AT: 
 -> Cantidad de clientes: 56,000 
 -> Energía facturada: 5.9804 TWh 
Clientes No Residenciales LibreDx: 
 -> Cantidad de clientes: 426 
 -> Energía facturada: 7.097 TWh 
-------------------------------------------- 
Cantidad de UM para TD: 
 -> UM 1F para TD: 22,259 
 -> UM 3F 1S para TD: 28,854 
 -> UM 3F 2S para TD: 93,120 



## Se guardan los cambios realizados para su posterior uso en '*Costo_de_implementacion_SMI_Chile.ipynb*'

In [19]:
df_caracterizacion_distribucion.rename(columns={"Superficie_efectiva_km2": "Superficie efectiva [km2]",
                                                "Tipo de facturacion": "Tipo de facturación"}, inplace=True)

df_caracterizacion_distribucion.to_excel("./Salidas/Caracterizacion_Dx_Dic2024_comuna (CNE).xlsx", index=False)